In [1]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from googleapiclient.discovery import build
import os
import sys
import re
import json
import time

import numpy as np
import pandas as pd
from tqdm import tqdm

from youtube_transcript_api import YouTubeTranscriptApi
from get_timestamps import get_timestamps, get_text
from yt_dlp import YoutubeDL
import whisper
from transformers import pipeline
from datasets import load_dataset, Audio

import json
import os
import pandas as pd
from tqdm import tqdm
import sys
import numpy as np
from transformers import pipeline
import datetime
import random
import math


# URL TO PERFORM INFERENCE ON
video_url = 'https://www.youtube.com/watch?v=BKpmAGZQetc'


/home/mrbean/.conda/envs/youtube_timestamps/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def download_audio(url, output_file):
    # Download audio file from a youtube url using yt-dlp
    ffmpeg_loc = "C:/Users/aladd/Desktop/Shortcuts/ffmpeg/bin/ffmpeg.exe"

    """
    Download audio file from a youtube url using yt-dlp
    """
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": output_file,
        "ffmpeg_location": ffmpeg_loc,
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "wav",
                "preferredquality": "196",
            }
        ],
    }
    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def convert_srt_to_json(srt):
    """ 
    From YouTubeTranscriptAPI we get a srt file,
    this function converts it to a json file and 
    formats it in the same structure as Whisper output.
    """
    json_list = []

    for dict in srt:
        start_time = dict["start"]
        end_time = dict["start"] + dict["duration"]
        json_list.append(
            {
                "start": start_time,
                "end": end_time,
                "text": dict["text"],
            }
        )
    
    result = {"segments":json_list}
    return result
    
def download_transcript_from_API(video_id):
    """Download transcript from YouTube API"""
    srt = YouTubeTranscriptApi.get_transcript(video_id)
    transcript = convert_srt_to_json(srt)
    return transcript

def download_transcript_using_whisper(video_id, output_file="audio/audio.wav"):
    # make video_id into youtube url
    url = f"https://www.youtube.com/watch?v={video_id}"

    # delete audio.wav if it exists
    if os.path.exists(output_file):
        os.remove(output_file)
        
    # download audio file
    download_audio(url, output_file)

    # transcript using whisper
    # time how long it takes 
    start = time.time()
    result = model.transcribe(output_file, verbose=True)
    end = time.time()
    print(f"Whisper model took {end-start} seconds to transcribe")
    return result

In [3]:
model_name = "facebook/bart-large-cnn"
summarizer = pipeline(
    "summarization",
    model=model_name,
    tokenizer=model_name,
    device="cuda:0",
)

model = whisper.load_model("medium.en")

In [4]:
def summarize_transcript(data):
    subset_texts = []
    text = ""
    start_time = data["segments"][0]["start"]
    idx = 0

    while idx < len(data["segments"]):
        segment = data["segments"][idx]
        segment_start_time = segment["start"]
        segment_end_time = segment["end"]
        segment_text = segment["text"]

        # add segment_text to text
        text += " " + segment_text.strip() if text != "" else segment_text.strip()

        # check if text is longer
        if len(text) > 1500:
            subset_texts.append([start_time, segment_end_time, text])
            start_time = segment_end_time
            text = ""
            idx -= 5
        idx += 1
    
    # add the last text
    subset_texts.append([start_time, segment_end_time, text])

    #print("Starting summarization")
    summary_dict = summarizer(
        [subset[2] for subset in subset_texts],
        max_length=100,
        min_length=0,
    )

    return summary_dict, subset_texts

In [5]:
video_id = video_url.split("=")[1]

try:    
    result = download_transcript_from_API(video_id)
except:
    print("Failed to download transcript from API, using Whisper instead")
    result = download_transcript_using_whisper(video_id)
summary_dict, subset_texts = summarize_transcript(result)

dataset = []
start_end_times = []
overlap_size = 1
summarized_text_string = ""

# take out summary_text from summary_dict
summaries = [summary_d["summary_text"] for summary_d in summary_dict]
for subset, summary_d in zip(subset_texts, summary_dict):
    start_end_times.append([subset[0], subset[1]])

# take out 10 of summary_dict at a time, with overlap of 1
summaries = [summaries[i : i + 10] for i in range(0, len(summaries), 10 - overlap_size)]
start_end_times = [start_end_times[i : i + 10] for i in range(0, len(start_end_times), 10 - overlap_size)]

# for each in summaries join with 1., 2., 3. etc
for i, summary in enumerate(summaries):
    summary = [f"{i+1}. {s}" for i, s in enumerate(summary)]
    summary = "\n".join(summary)
    summary += "\n\n###\n\n"
    dataset.append({"prompt":summary})

In [6]:
import openai
ft_model = 'babbage:ft-personal:timestamps-v2-2022-12-10-19-16-54'

keywords = []
openai.api_key = os.environ['API_KEY']

prev_ans = ""
for idx in range(len(dataset)):
    prompt = dataset[idx]["prompt"]
    prompt += f" 1. {prev_ans}\n2" if len(prev_ans) > 0 else ""
    keywords += [prev_ans] if len(prev_ans) > 0 else []
    res = openai.Completion.create(model=ft_model, prompt=prompt, max_tokens=256, temperature=0.0, presence_penalty=0.5, frequency_penalty=0.5, stop=["###"])

    print(prompt + res['choices'][0]['text'])
    print("\n\n")
    
    for each in res['choices'][0]['text'].split("\n"):
        if isinstance(each, str) and len(each) > 0:
            keywords.append(each.split(".")[1].strip())
    
    prev_ans = keywords[-1]

1. Liver King believes the modern world has made made men soft and it's his duty to empower the masses to become Jack high testosterone metabolically and cognitively Peak primals with a higher quality of life. He does it through as a viral of content as possible he walks the walk and shows off the polar extreme of ancestral living.
2. Liver King's content includes being shirtless essentially in perpetuity doing hardship constantly via heavy weight training and unorthodox difficult fitness challenges as well as showing off the ancestrally consistent median organs he eats huge quantities of. This includes heart raw testicles liver Etc and flying around the world to share his beliefs.
3. Liver King has been very candid about the success of his portfolio of supplement companies. His success draws a greater level of scrutiny to his claims of being natural and his brand.
4. Liver King says he's always told the truth. Joe Rogan criticized him for using steroids. The liver King has never taken

In [7]:
start_end_times = [item for sublist in start_end_times for item in sublist]

In [8]:
# make a unique list of keywords and their start and end times in order
unique_keywords = []
for keyword, (start, end) in zip(keywords, start_end_times):
    if len(unique_keywords) == 0:
        unique_keywords.append([keyword, [start, end]])
    elif keyword != unique_keywords[-1][0]:
        unique_keywords.append([keyword, [start, end]])
    else:
        unique_keywords[-1][1][1] = end

print(unique_keywords)




[['Intro', [0.0, 249.35999999999999]], ['Truths About Liverkings Content & Brand', [249.35999999999999, 456.599]], ['Steroids Or Supplements?', [456.599, 664.38]], ['How To Spot a Pseudo-Primal', [664.38, 1024.02]], ["The Liver King's 9 Ancestral Tenets", [1024.02, 1282.9189999999999]], ['Brian Johnson & Liver King', [1282.9189999999999, 1978.44]], ['Email 1 - January 2021', [1978.44, 2057.46]], ['Email 2 - January 2021', [2057.46, 2137.56]], ['Email 3 - February 2021', [2137.56, 2224.02]], ["Brian's protocol for boosting GH & IGF-1", [2224.02, 2547.48]], ['Closing Thoughts', [2547.48, 2905.92]], ['Current Diet & Carnivore Style', [2905.92, 3195.599]], ["Brian Johnson's HGH Story", [3195.599, 3612.98]]]


In [9]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")


In [10]:
# iterate through every segment and print start, end, text
time_stamps = []
print(len(result["segments"]))
for segment in tqdm(result["segments"]):
    sequence_to_classify = segment['text']
    start_seg = segment['start']
    end_seg = segment['end']
    # take out 3 keywords from unique_keywords that are closest to start_time and end_time
    i = np.where([start_seg>=start_key for keyword, (start_key, end_key) in unique_keywords])[0].max()
    # add i-1, i, i+1 to candidate_labels if they exist
    candidate_labels = []
    if i-1 >= 0:
        candidate_labels.append(unique_keywords[i-1][0])
    candidate_labels.append(unique_keywords[i][0])
    if i+1 < len(unique_keywords):
        candidate_labels.append(unique_keywords[i+1][0])
    res = classifier(sequence_to_classify, candidate_labels)
    # get max label and add to time_stamps
    max_label = res['labels'][res['scores'].index(max(res['scores']))]
    time_stamps.append([max_label, start_seg])

1663


100%|██████████| 1663/1663 [07:08<00:00,  3.88it/s]


In [11]:
time_stamps[0][1] = 0

In [12]:
for each in time_stamps:
    if type(each[1]) == str:
        # convert hh:mm:ss to seconds
        each[1] = sum(x * int(t) for x, t in zip([3600, 60, 1], each[1].split(":")))

In [13]:
# iterate through time_stamps and only keep the first instance of each label each time it occurs
# iterate through time_stamps and print
def clean_time_stamps(time_stampo):
    time_stamps = time_stampo.copy()
    unique_time_stamps = []
    for time_stamp in time_stamps:
        if len(unique_time_stamps) == 0:
            unique_time_stamps.append(time_stamp)
            
        elif time_stamp[0] != unique_time_stamps[-1][0]:
            unique_time_stamps.append(time_stamp)
    
     # go through unique_time_stamps and remove any that are within 30 seconds of the next one, remove repeated labels 
    i = 0
    while i < len(unique_time_stamps)-1:
        if unique_time_stamps[i+1][1] - unique_time_stamps[i][1] < 30:
            unique_time_stamps.pop(i+1)
        elif unique_time_stamps[i+1][0] in [t[0] for t in unique_time_stamps[:i+1]]:
            unique_time_stamps.pop(i+1)
        else:
            i += 1

    # convert unique_time_stamps to h:m:s format and remove ms
    for i, time_stamp in enumerate(unique_time_stamps):
        unique_time_stamps[i][1] = str(datetime.timedelta(seconds=time_stamp[1])).split('.')[0]
    print(f'Second clean: {unique_time_stamps}')

    # print the unique_time_stamps in nice format with time and label
    for time_stamp in unique_time_stamps:
        print(f"{time_stamp[1]}: {time_stamp[0]}")

clean_time_stamps(time_stamps)

Second clean: [['Intro', '0:00:00'], ['Truths About Liverkings Content & Brand', '0:00:35'], ['Steroids Or Supplements?', '0:04:24'], ['How To Spot a Pseudo-Primal', '0:08:13'], ["The Liver King's 9 Ancestral Tenets", '0:11:04'], ['Brian Johnson & Liver King', '0:17:06'], ['Email 1 - January 2021', '0:27:30'], ['Email 2 - January 2021', '0:33:57'], ['Email 3 - February 2021', '0:34:38'], ["Brian's protocol for boosting GH & IGF-1", '0:35:37'], ['Closing Thoughts', '0:37:04'], ['Current Diet & Carnivore Style', '0:43:50'], ["Brian Johnson's HGH Story", '0:52:22']]
0:00:00: Intro
0:00:35: Truths About Liverkings Content & Brand
0:04:24: Steroids Or Supplements?
0:08:13: How To Spot a Pseudo-Primal
0:11:04: The Liver King's 9 Ancestral Tenets
0:17:06: Brian Johnson & Liver King
0:27:30: Email 1 - January 2021
0:33:57: Email 2 - January 2021
0:34:38: Email 3 - February 2021
0:35:37: Brian's protocol for boosting GH & IGF-1
0:37:04: Closing Thoughts
0:43:50: Current Diet & Carnivore Style
0

In [ ]:
s1 = """0:00:00: What is a language model?
0:00:51: Transformers architecture
0:05:51: How to train the model?
0:09:35: Free and open source models"""


s2 = """0:00:00: The Hyperloop
0:00:32: Elon Musk and AI
0:09:56: Barack Obama's views on AI
0:30:47: Artificial intelligence
0:34:57: Extending the human timeline in the universe
0:43:38: Tesla game
0:58:13: Flat Earth theory
1:08:56: Flying cars
1:12:08: Electric cars vs gasoline powered ones
1:18:14: Justice system
1:34:50: Worst person in the world scenario
1:40:50: Texas
1:43:09: Huawei
1:47:13: AI and society
1:53:00: Tesla, SpaceX, Uber, etc
1:53:49: Solar panels
2:01:10: Charging system
2:05:31: Case for the Tesla Model 3
2:15:19: Social media
2:36:26: Elon Musk and Steve Jobs"""

s3 = """0:00:00: Intro
0:01:29: Red Lipstick & Holiday Celebrations
0:02:17: Deepika’s Story
0:05:21: Social Media Insecurities
0:12:10: Beauty & Identity, 10 Years Later
0:21:24: Funding Deepika’s New Brand
0:29:07: Advice for Young People
0:33:30: How do you normally plan your day?
0:38:46: Book that has been very helpful to you: “Burn Rate” by Andy Dunn, “The Truth About Men” by Shailja Gollapunjia, “The Truth About Men” by Shailja Gollapunjia, “Burn Rate” by Andy Dunn, “The Truth About Men” by Shailja Gollapunjia
0:45:22: Book that has been very helpful to you: “Burn Rate” by Andy Dunn, “The Truth About Men” by Shailja Gollapunjia, “Burn Rate” by Andy Dunn, “The Truth About Men” by Shailja Gollapunjia
0:47:30: Deepika’s new beauty brand, Live Tinted
0:48:36: What is the most memorable moment of The Today Show?"""

s4 = """0:00:00: Intro
0:00:35: Truths About Liverkings Content & Brand
0:04:24: Steroids Or Supplements?
0:08:13: How To Spot a Pseudo-Primal
0:11:04: The Liver King's 9 Ancestral Tenets
0:17:06: Brian Johnson & Liver King
0:27:30: Email 1 - January 2021
0:33:57: Email 2 - January 2021
0:34:38: Email 3 - February 2021
0:35:37: Brian's protocol for boosting GH & IGF-1
0:37:04: Closing Thoughts
0:43:50: Current Diet & Carnivore Style
0:52:22: Brian Johnson's HGH Story"""


s5 = """0:00:00: Acoustic Treatment
0:00:44: Diffusion absorbers
0:03:29: Measuring the room
0:07:30: Final thoughts and comments"""


In [17]:
titles = ['Machine Learning for Natural Language Processing: Transformer Language Models', ' Joe Rogan Experience #1169 - Elon Musk',
    'Deepica Mutyala | The New Standards of Beauty | Talks at Google',  'The Liver King Lie', 'Pro-Grade Acoustic Treatment' 
    ]

links = ['https://www.youtube.com/watch?v=hcU62ItWKA0', 'https://www.youtube.com/watch?v=ycPr5-27vSI',
 'https://www.youtube.com/watch?v=mfeynfAY9kw', 'https://www.youtube.com/watch?v=BKpmAGZQetc',
 'https://www.youtube.com/watch?v=bB4ihxp5ALY']


timestamps = [s1, s2, s3, s4, s5]

In [1]:
import pandas as pd
title_and_link = [f'{title} \\ {link}' for title, link in zip(titles, links)]
print(title_and_link)
df = pd.DataFrame({'Youtube link and title': title_and_link, 'timestamps': timestamps})
df

NameError: name 'titles' is not defined